# Behind the pipeline (TensorFlow)

Emulating sentient analysis

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
!pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 4.0 MB 46.7 MB/s 
     |████████████████████████████████| 77 kB 2.6 MB/s 
     |████████████████████████████████| 212 kB 68.2 MB/s 
     |████████████████████████████████| 136 kB 55.4 MB/s 
     |████████████████████████████████| 1.1 MB 68.9 MB/s 
     |████████████████████████████████| 127 kB 67.7 MB/s 
     |████████████████████████████████| 271 kB 42.7 MB/s 
     |████████████████████████████████| 144 kB 47.8 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 880 kB 55.6 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 6.6 MB 50.5 MB/s 
     |████████████████████████████████| 1.2 MB 58.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8a35cdfbca160a511d149ea0f68be58529a8478950228f1a5d1a6309ae0b3e9d
  Stored in directory: /root/.cac

Sentiment Analysis using Transformers

In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    ["I've been waiting for this course my whole life.", "I hate this so much!"]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9956169128417969},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

# Pipeline steps

Preprocessing with a tokenizer

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [4]:
raw_inputs = ["I've been waiting for this course my whole life.", "I hate this so much!"]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 14), dtype=int32, numpy=
array([[ 101, 1045, 1005, 2310, 2042, 3403, 2005, 2023, 2607, 2026, 2878,
        2166, 1012,  102],
       [ 101, 1045, 5223, 2023, 2061, 2172,  999,  102,    0,    0,    0,
           0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 14), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


Download the pretrained model (general)

Note :- This section is an example of a general structure and won't be used dirctly in the pipeline

In [5]:
from transformers import TFAutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertModel: ['dropout_19', 'classifier', 'pre_classifier']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [6]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 14, 768)


The outputs shape is defined as :- 

*   Batch size: The number of sequences processed at a time.
*   Sequence length: The length of the numerical representation of the sequence.
*   Hidden size: The vector dimension of each model input.

Download the pretrained model

Note :- This section is the actual usage. We are using a specific class of automodel specific to sequence classification.

In [7]:
from transformers import TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(inputs)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_38']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print(outputs.logits.shape)

(2, 2)


In [11]:
print(outputs.logits)

tf.Tensor(
[[-2.721174   2.7044277]
 [ 4.169232  -3.3464477]], shape=(2, 2), dtype=float32)


Converting logits to probability using softmax

In [12]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[4.3831193e-03 9.9561691e-01]
 [9.9945587e-01 5.4418371e-04]], shape=(2, 2), dtype=float32)


To make sense of the probability values, we can call the id2label class to get what each value means

In [13]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}